<a href="https://colab.research.google.com/github/marcocaggioni/remote-lab/blob/main/remote_lab_timelapse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Remote lab

In [43]:
!pip install ipywebrtc
!pip install humanize --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import io
import time

import PIL.Image
import PIL.ImageFilter
from IPython.display import FileLink, display
from ipywebrtc import CameraStream, ImageRecorder
from ipywidgets import Button, Output, VBox, Text

from google.colab import output
output.enable_custom_widget_manager()

from IPython.display import HTML
from base64 import b64encode
import os

In [45]:
from IPython.core.display import clear_output
camera = CameraStream(
    constraints={
        "facing_mode": "environment",
        "audio": False,
        "video": {"width": 640, "height": 480},
    }
)

In [46]:
numimage_wid=Text(description='Im number', value='10')
interval_wid=Text(description='Interval s', value='1')
exp_name_wid=Text(description='exp name', value='test')

In [47]:
recorder = ImageRecorder(stream=camera)
out = Output()

numImage=0

def savePicture(_):
    with out:
        global numImage
        global numImages
        numImages=int(numimage_wid.value)
        
        if numImage >= numImages:
            return

        interval=int(interval_wid.value)
        
        recorder.recording = True
        out.append_stdout(f"{exp_name_wid.value}" + str(numImage) + " captured \n")
        recorder.save(exp_name_wid.value + str(numImage) + ".png")
        numImage = numImage + 1
        time.sleep(interval)


recorder.image.observe(savePicture, names=["value"])

In [48]:
l1=[0,1,2,3,4,5]
l2=[5,4,3,2,1,0]


list1, list2 = zip(*sorted(zip(l1, l2)))
print(list1)
print(list2)

(0, 1, 2, 3, 4, 5)
(5, 4, 3, 2, 1, 0)


In [54]:
import datetime
from pathlib import Path
#from IPython.display import Video
import os

import numpy as np
from moviepy.editor import *
from PIL import Image, ImageDraw, ImageFont
import humanize

video_out = Output()


def make_video(object):

    files = list(Path("./").glob(f"{exp_name_wid.value}*.png"))
    times = [datetime.datetime.fromtimestamp(file.stat().st_mtime) for file in files]
    times = list(map(lambda x: x - times[0], times))

    times, files = zip(*sorted(zip(times, files)))


    font = ImageFont.truetype("/usr/share/fonts/truetype/humor-sans/Humor-Sans.ttf", 40)

    clips = []
    for file, t in zip(files, times):
        img = Image.open(file)
        time_stamp = humanize.time.precisedelta(t, minimum_unit="seconds")

        ImageDraw.Draw(img).text((0, 0), time_stamp,font=font)
        clip = ImageClip(np.array(img), duration=0.1)
        clips.append(clip)

    concat_clip = concatenate_videoclips(clips, method="compose")
    
    concat_clip.write_videofile(f"{exp_name_wid.value}.mp4", fps=24)


    with video_out:
        mp4 = open(f"{exp_name_wid.value}.mp4",'rb').read()
        data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
        display(HTML("""
        <video width=400 controls>
              <source src="%s" type="video/mp4">
        </video>
        """ % data_url))

make_video_button = Button(description="Make Video")
make_video_button.on_click(make_video)

# User input

In [50]:
camera

CameraStream(constraints={'facing_mode': 'environment', 'audio': False, 'video': {'width': 640, 'height': 480}…

In [51]:
display(VBox([exp_name_wid,numimage_wid,interval_wid]))

In [52]:
VBox([recorder, out])

In [55]:
VBox([make_video_button, video_out])

[MoviePy] >>>> Building video test.mp4
[MoviePy] Writing video test.mp4


100%|██████████| 24/24 [00:00<00:00, 46.14it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test.mp4 



In [ ]:
!rm *.png